## The analysis pipeline is wrapped into a Python package: probacseq_toolkit

In [1]:
%load_ext autoreload
%autoreload 2

import os
import probacseq_toolkit

## 1. Map probes to the E.coli reference genome

1. Map probes to the E.coli reference with BWA software
2. Identify and merge overlapping probes

In [2]:
%%time

probe_file = "./data/probes_sequence.csv"
bwa_path = "/data/programs/bwa-0.7.17/bwa"
reference_genome = "./data/E_coli_reference/Escherichia_coli_w_gca_000184185.ASM18418v1.dna.chromosome.Chromosome.fa"

probe_df = probacseq_toolkit.map_probes(probe_file, bwa_path, reference_genome, merge_overlapping_probes=True)
probe_df

[bwa_index] Pack FASTA... 0.05 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 1.30 seconds elapse.
[bwa_index] Update BWT... 0.03 sec
[bwa_index] Pack forward-only FASTA... 0.03 sec
[bwa_index] Construct SA from BWT and Occ... 0.58 sec
[main] Version: 0.7.17-r1188
[main] CMD: /data/programs/bwa-0.7.17/bwa index /homes8/jingyuan/Projects/bacteria_sc_multiplex/data/E_coli_reference/Escherichia_coli_w_gca_000184185.ASM18418v1.dna.chromosome.Chromosome.fa
[main] Real time: 2.439 sec; CPU: 1.998 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 19550 sequences (1160650 bp)...
[M::mem_process_seqs] Processed 19550 reads in 0.451 CPU sec, 0.451 real sec
[main] Version: 0.7.17-r1188
[main] CMD: /data/programs/bwa-0.7.17/bwa mem ./data/E_coli_reference/Escherichia_coli_w_gca_000184185.ASM18418v1.dna.chromosome.Chromosome.fa ./data/E_coli_reference/probe.fa
[main] Real time: 0.540 sec; CPU: 0.488 sec


./data/mapped_probes.csv
CPU times: user 19 s, sys: 126 ms, total: 19.1 s
Wall time: 22.3 s


,probe,gene,strand,position,sequence
0,aaeA_1,aaeA,-,3601479,CAGCTTAGTTTCTTCCATATAGGCCAGTACATAGAAGGAGTTCTGT...
1,aaeA_2,aaeA,-,3601861,AGTATCTGTCCTTTTTTCACCAGCTGGTTATCATGAACATTCACCT...
2,aaeA_3,aaeA,-,3602022,GGCCAGAATGACTAATACGACCGTGATGGCCGTACGGGAGAATTTT...
0,aaeB_1,aaeB,-,3599466,GAAATTTCGGCAAATCCCCTGGGAACTTATTCATCAGCAAAAACAG...
1,aaeB_2,aaeB,-,3599701,AAACTGACTGAAATGGAAAGTCATCGGGTTATCCAGCACGATAATA...
...,...,...,...,...,...
0,zur_1,zur,-,4525442,CTGATCGAACAGATGACAGAGCACATAACTGTTGGTGGATTCCACC...
0,zwf_1,zwf,-,2083199,AGATGCGTCTGATTAAAGGTTTCTGAATAGCTCAGATCCAGCTTGG...
1,zwf_2,zwf,-,2083369,AAACAGATTCAGTTCAGGTGTTTTGAAATAGACCACGACTTCAGAA...
2,zwf_3,zwf,-,2083897,CACCGTTTCTTTACCAAGATAGTGGTCGATACGGTAAACCTGGCAC...


## 2. Make a custom reference genome by stitching the probes together.

1. Sort the probes by their position on the reference E.coli
2. Convert the probes to its reverse complement if they are on the reverse strand
3. Add NNNN paddings to the front and end of each probe

In [3]:
%%time

probe_ref_dir = "./data/probe_reference/"
output_file_prefix = "E.coli"
probacseq_toolkit.make_fa_and_gtf_reference(probe_df, probe_ref_dir, output_file_prefix,
                                            fwd_primer = "NNNNNNN", rev_primer = "NNNNNNN")

CPU times: user 9.06 s, sys: 12.7 ms, total: 9.07 s
Wall time: 9.18 s


## 3. Make 10x reference index with cell ranger

In [4]:
%%time

path_to_cell_ranger = "/data/programs/cellranger-7.1.0/bin/cellranger"
genome_name = "E.coli_probes"
fa_gtf_prefix = "%s%s"%(probe_ref_dir,output_file_prefix)
output_dir = probe_ref_dir
probacseq_toolkit.build_reference_index_with_cell_ranger(path_to_cell_ranger, genome_name,\
                                            fa_gtf_prefix, output_dir)

/data/programs/cellranger-7.1.0/bin/cellranger mkref --genome=E.coli_probes --fasta=/homes8/jingyuan/Projects/bacteria_sc_multiplex/data/probe_reference/E.coli.fasta --genes=/homes8/jingyuan/Projects/bacteria_sc_multiplex/data/probe_reference/E.coli.gtf
['/data/programs/cellranger-7.1.0/bin/rna/mkref', '--genome=E.coli_probes', '--fasta=/homes8/jingyuan/Projects/bacteria_sc_multiplex/data/probe_reference/E.coli.fasta', '--genes=/homes8/jingyuan/Projects/bacteria_sc_multiplex/data/probe_reference/E.coli.gtf']
CPU times: user 3.32 ms, sys: 11.9 ms, total: 15.2 ms
Wall time: 1.36 s


Destination reference folder already exists: /homes8/jingyuan/Projects/bacteria_sc_multiplex/data/probe_reference/E.coli_probes
Please delete and start again.


## 4. Segment reads to each component
1. Identify each component with regular expression. We'll need a csv file.
2. Merge probe UMI with 10X UMI
3. Demux the data by sample_identifier. 
4. The unidentified fastq file contains read that are either: (1) anchors can't be aligned to the read. (2) Sample_identifier is not in the white list

In [5]:
import pandas as pd

construct_df = pd.read_csv("./data/construct.csv")
construct_df

,segment_name,length,sequence
0,anchor,22,CGCTAGTATGGATAGCCGTGTC
1,sample,4,AAGG|GGAA|TTCC|CCTT
2,umi,12,.
3,anchor,15,GACCCGGTCCATACA
4,probe,30-70,.
5,anchor,19,CATAGTTTCTTCGAGCAAG


In [6]:
construct_regex = probacseq_toolkit.make_construct_regex_pattern(construct_df, insertion=1, deletion=1,
                                 substitution=3, error=3)
construct_regex

'(?P<anchor>CGCTAGTATGGATAGCCGTGTC{i<=1,d<=1,s<=3,e<=3})(?P<sample>(AAGG|GGAA|TTCC|CCTT))(?P<umi>.{12})(?P<anchor>GACCCGGTCCATACA{i<=1,d<=1,s<=3,e<=3})(?P<probe>.{30,70})(?P<anchor>CATAGTTTCTTCGAGCAAG{i<=1,d<=1,s<=3,e<=3})'

In [7]:
%%time

r1_fastq = "./data/30-844302426/00_fastq/SBS_R1_001.fastq.gz"
r2_fastq = "./data/30-844302426/00_fastq/SBS_R2_001.fastq.gz"
segment_fq_dir = "./data/30-844302426/segmented_fastq/"
file_prefix = "SBS"
sample_identifiers = ["AAGG","GGAA","TTCC","CCTT"]

probacseq_toolkit.segment_fastq_multiplex(construct_regex, r1_fastq, r2_fastq, use_reverse_complement=True,
                            sample_identifiers=sample_identifiers, output_dir=segment_fq_dir, file_prefix=file_prefix)

CPU times: user 27.3 s, sys: 99.8 ms, total: 27.4 s
Wall time: 28.2 s


## 5. Align reads to the reference genome with cell ranger

In [8]:
%%time

path_to_cell_ranger = "/data/programs/cellranger-7.1.0/bin/cellranger"
transcriptome = "/homes8/jingyuan/Projects/bacteria_sc_multiplex/data/probe_reference/E.coli_probes"
for sample_idx in sample_identifiers:
    id = sample_idx
    fastq_dir = os.path.abspath("%s/%s/"%(segment_fq_dir,sample_idx))
    localcores = 6
    localmem = 32
    probacseq_toolkit.run_cellranger_alignment(path_to_cell_ranger, id, transcriptome,
                             fastq_dir, localcores=localcores, localmem=localmem)

This is the cell ranger command:
/data/programs/cellranger-7.1.0/bin/cellranger count --id=AAGG --chemistry=threeprime --transcriptome=/homes8/jingyuan/Projects/bacteria_sc_multiplex/data/probe_reference/E.coli_probes --fastqs=/homes8/jingyuan/Projects/bacteria_sc_multiplex/data/30-844302426/segmented_fastq/AAGG --localcores=6 --localmem=32
Martian Runtime - v4.0.10
2023-05-10 23:06:50 [runtime] Reattaching in local mode.
Serving UI at http://lynx:41145?auth=qUP-jl2FvxhRIkWAy0BkJHXJ8HwPaK1g3whe7Qm6xek


Outputs:
- Run summary HTML:                         /homes8/jingyuan/Projects/bacteria_sc_multiplex/AAGG/outs/web_summary.html
- Run summary CSV:                          /homes8/jingyuan/Projects/bacteria_sc_multiplex/AAGG/outs/metrics_summary.csv
- BAM:                                      /homes8/jingyuan/Projects/bacteria_sc_multiplex/AAGG/outs/possorted_genome_bam.bam
- BAM BAI index:                            /homes8/jingyuan/Projects/bacteria_sc_multiplex/AAGG/outs/possorted_ge

Pipestance completed successfully!

2023-05-10 23:07:27 Shutting down.
Saving pipestance info to "CCTT/CCTT.mri.tgz"
This is the cell ranger command:
/data/programs/cellranger-7.1.0/bin/cellranger count --id=unidentified --chemistry=threeprime --transcriptome=/homes8/jingyuan/Projects/bacteria_sc_multiplex/data/probe_reference/E.coli_probes --fastqs=/homes8/jingyuan/Projects/bacteria_sc_multiplex/data/30-844302426/segmented_fastq/unidentified --localcores=6 --localmem=32
Martian Runtime - v4.0.10
2023-05-10 23:07:30 [runtime] Reattaching in local mode.
Serving UI at http://lynx:46079?auth=MRF_0aLMRZEGEnVDFT5uWW84koqLFVcswuFAkZHoZR8


Outputs:
- Run summary HTML:                         /homes8/jingyuan/Projects/bacteria_sc_multiplex/unidentified/outs/web_summary.html
- Run summary CSV:                          /homes8/jingyuan/Projects/bacteria_sc_multiplex/unidentified/outs/metrics_summary.csv
- BAM:                                      /homes8/jingyuan/Projects/bacteria_sc_multiplex/